In [1]:
import os
os.environ['SPARK_HOME']='/home/envmodules/lib/spark-2.2.0-bin-hadoop2.7/'

import findspark
findspark.init()

In [2]:
# Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [3]:
spark = SparkSession.builder.appName('733').getOrCreate()

In [4]:
# Using the integrated file to start working on
integrated_df = spark.read.csv('/user/vcs/annual_compustat.csv',header=True,inferSchema=True)

In [5]:
integrated_df.first()

Row(gvkey=1000, datadate=19611231, fyear=1961, indfmt='INDL', consol='C', popsrc='D', datafmt='STD', tic='AE.2', cusip='000032102', conm='A & E PLASTIK PAK INC', acctchg=None, acctstd=None, acqmeth=None, adrr=None, ajex=3.3418, ajp=1.0, bspr=None, compst=None, curcd='USD', curncd='USD', currtr=1.0, curuscn=None, final='Y', fyr=12, ismod=None, ltcm=None, ogm=None, pddur=12, scf=None, src=None, stalt=None, udpl=None, upd=3, apdedate=None, fdate=None, pdate=None, acchg=None, acco=None, acdo=None, aco=None, acodo=None, acominc=None, acox=None, acoxar=None, acqao=None, acqcshi=None, acqgdwl=None, acqic=None, acqintan=None, acqinvt=None, acqlntal=None, acqniintc=None, acqppe=None, acqsc=None, act=None, adpac=None, aedi=None, afudcc=None, afudci=None, aldo=None, am=None, amc=None, amdc=None, amgw=None, ano=None, ao=None, aocidergl=None, aociother=None, aocipen=None, aocisecgl=None, aodo=None, aol2=None, aoloch=None, aox=None, ap=None, apalch=None, apb=None, apc=None, apofs=None, aqa=None, aqc

In [ ]:
# integrated_df.count()

In [ ]:
ls = ['ACQ', 'CONM', 'ACCHG', 'ACO', 'ACOMINC', 'ACQCSHI', 'ACQGDWL', 'ACQINTAN', 'ACQLNTAL', 'ACT', 'AMGW','AQ','AP','AQC','ARC','AT','AU','AUOP','AUOPIC','BKVLPS','CAPX','CDVC','CEQ','CGA','CHE','CHECH','CI','CIMII','CLD2','CLD3','CLD4','CLD5','CLT','COGS','CSHI','CURNCD','DCOM','DCPSTK','DCVT','DEPC','DLTT','DM','DO','DP','DPACLS','DRC','DRLT','DV','DVC','DVPA','DVPDP','DVPSP_F','EBIT','EBITDA','EMOL','EMP','EPSFI','EPSPI','ESOPCT','ESOPDLT','EXCHG','EXRE','FATB','FATE','FATN','FATL','FATO','FCA','FEL','FIC','FINCF','FOPT','FYEAR','FYR','GDWL','GDWLAM','GDWLIA','GLA','GP','GWO','HEDGEGL','ICAPT','IDBFLAG','IDIIS','IDILC','IDIT','INTAN','INTC','INVCH','INCFG','INVOFS','INVRM','INVT','INVWIP','IPODATE','ISEQ','ISEQC','ISEQM','ISFI','ISGR','ISGU','ITCB','IVCH','IVNCF','LCT','LCUACU','LIFR','LLRCI','LLWOCI','LT','MIB','MIBT','MII','MKVALT','NAICS','NI','NIINT','NIINTPFC','NIINTPFP','NOPIO','NP','NPANL','NPAT','NRTXT','OANCF','OB','OIADP','OIBDP','OPEPS','OPINI','OPITI','OPREPSX','OPTCA','OPTEX','OPTEXD','OPTFVGR','OPTGR','OPTPRCCA','OPTPRCEX','OPTPRCEY','OPTPRCGR','OPTPRCWA','OPTVOL','PCL','PI','PIFO','PLL','PNCA','PNCIA','PPEGT','PPENB','PPENC','PPENLS','PPENNR','PPENT','PPEVO','PPEVR','PRCC_F','PRCH_F','PRICAN','PRIROW','PRIUSA','PRODV','PRSTKCC','PRSTKPC','PSTKC','PVCL','PVO','PVPL','PVT','RANK','RCA','RDIP','RE','REA','REAJO','RECCH','RECT','RECTA','REUNA','REVT','RLL','RMUM','RPAG','RVLRV','RVTI','SALE','SALEPFC','SALEPFP','SCO','SCSTKC','SEQ','SIC','SIV','SPCE','SPCINDCD','SPCSECCD','SPCSRC','SPI','SPPE','SPPIV','SPSTKC','SRET','SSNP','STBO','STIO','STKCO','STKO','TDC','TEQ','TFVA','TFVCE','TFVL','TIC','TLCF','TRANSA','TSA','TSTKC','TSTKP','TXC','TXDB','TXDBA','TXDBCA','TXDBCL','TXFED','TXFO','TXPD','TXR','TXT','TXTUBXINTBS','TXTUBXINTIS','VPAC','VPO','WCAP','WCAPC','WCAPCH','WDA','XACC','XAD','XAGO','XAGT','XCOM','XDP','XEQO','XI','XIDO','XINST','XINT','XINTD','XINTOPT','XLR','XOPR','XOPRAR','XPP','XPR','XRD','XRENT','XS','XSGA','XSTF','XSTFO','XSTFWS','XT']
ls = [x.lower() for x in ls]
annual_filtered = integrated_df.select([c for c in integrated_df.columns if c in ls])

In [ ]:
integrated_df.columns

In [ ]:
annual_filtered.select('acq').distinct().show()

In [ ]:
# Using nullcounts to filter columns to keep
nullcounts = annual_filtered.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in annual_filtered.columns])
nc = list(nullcounts.first())
# Extracting out an industrial segment and modelling on it instead of the whole dataset

# Services-packaged software category selection (from EDA)
services_prepacked_software = annual_filtered.filter(annual_filtered.sic == '7372')
print('Total records in integrated file: ', annual_filtered.count())
print(services_prepacked_software.show())
print('Number of records in Services-packaged software industrial category: ', services_prepacked_software.count())
# Reusing preprocessing steps implemented by Vincent
some_dict = {}
for x in services_prepacked_software.columns:
    some_dict[x] = 0
nwdf = services_prepacked_software.fillna(some_dict)
good_columns = []
for i in range(0, len(nc)):
    if nc[i] == 0:
        good_columns.append(i)
great_columns = [nwdf.columns[i] for i in good_columns]
great_columns.append('rea')
nwdf = nwdf.fillna(some_dict)
non_string_columns = [k for (k,v) in nwdf.dtypes if v != 'string']
nwdf_no_strings = nwdf.select(*non_string_columns)
feature_columns = [item for item in nwdf_no_strings.columns if item not in ['acq', 'acchg', 'rea', 'features']]

In [ ]:
nwdf_no_strings.limit(10).show()

In [ ]:
nwdf_no_strings.select('')

In [ ]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
final_df = assembler.transform(nwdf_no_strings)
final_final_df = final_df.drop(*feature_columns)
final_final_df = final_final_df.withColumn('boolean_label', final_final_df.rea != 0 & final_final_df.acchg == 0 & final_final_df.acq == 0 )
print('Class distribution: ', final_final_df.groupBy('boolean_label').count().show())
final_final_df = final_final_df.withColumn('label', final_final_df.boolean_label.cast('float'))
final_final_df = final_final_df.drop('rea').drop('boolean_label')
final_final_df = final_final_df.cache()
print(final_final_df.show())

In [ ]:
# Evaluators
evaluator = MulticlassClassificationEvaluator(metricName = 'accuracy')
eval = BinaryClassificationEvaluator()

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

In [ ]:
def balanceweights(df):
    minoritynum=df.filter(df['label']==1).count()
    datasetcount=df.count()
    balancingratio=(datasetcount-minoritynum)/datasetcount
    def weights(k):
        if(k==1):
            minweights=1.0*balancingratio
            return minweights
        else:
            majorityweigths = (1.0*(1-balancingratio))
            return majorityweigths
    awesome = udf(weights, FloatType())
    df = df.withColumn('weights', awesome(df['label']))
    print(df.show())
    return df
    

In [ ]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
final_final_df = final_final_df.drop('pcaFeatures')
pca = PCA(k=8, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(final_final_df)
final_final_df = model.transform(final_final_df) #.select("pcaFeatures")
final_final_df.show(truncate=True)

In [ ]:
model.explainedVariance

In [ ]:
len(model.explainedVariance)

In [ ]:
len([x for x in list(model.explainedVariance) if x > 0.001])

In [ ]:
final_final_df.first()

In [ ]:
new_final_df = balanceweights(final_final_df)  

In [ ]:
train, test = new_final_df.randomSplit([0.7, 0.3], seed=12345)

In [ ]:
logistic = LogisticRegression(regParam=0.1, labelCol="label", featuresCol='pcaFeatures', weightCol='weights')

In [ ]:
trained_model = logistic.fit(train)
res = trained_model.transform(test)
metrics = MulticlassMetrics(res.select(['label', 'prediction']).rdd)

In [ ]:
print('Accuracy on test set: ', evaluator.evaluate(res))
print('Precision on test data: ', metrics.precision())
print('Recall on test data: ', metrics.recall())
print('F1 Score on test data: ', metrics.fMeasure())
print('Area under ROC curve: ', eval.evaluate(res))

In [ ]:
res.show()

In [ ]:
res.groupby('prediction').count().show()

In [ ]:
res = res.withColumn('correct', res.label == res.prediction)

#positive class (misstatements)
true_positives_df = res.filter(res.prediction == 1.0).filter(res.label == 1.0)
ground_truth_positives_df = res.filter(res.label == 1.0)
misstatement_recall = true_positives_df.count()/ground_truth_positives_df.count()

new_all_predicted_positive_df = res.filter(res.prediction == 1.0)
misstatement_precision = true_positives_df.count()/new_all_predicted_positive_df.count()

#negative class (non misstatements)
true_negative_df = res.filter(res.prediction == 0.0).filter(res.label == 0.0)
ground_truth_negative_df = res.filter(res.label == 0.0)
non_misstatement_recall = true_negative_df.count()/ground_truth_negative_df.count()

new_all_predicted_negative_df = res.filter(res.prediction == 0.0)
non_misstatement_precision = true_negative_df.count()/new_all_predicted_negative_df.count()

In [ ]:
print('misstatement precision is {}, misstatement recall is {}'.format(misstatement_precision, misstatement_recall))
print('non-misstatement precision is {}, non-misstatement recall is {}'.format(non_misstatement_precision, non_misstatement_recall))